# Port code to Qiskit Serverless

The following example demonstrates how to port existing code to leverage Qiskit Serverless.
<Admonition type="note">
The following code assumes that you have saved your credentials. If you have not, follow the instructions in <LegacyContent>[Set up an IBM Quantum channel](/docs/guides/setup-channel#iqp)</LegacyContent><CloudContent>[Set up your IBM Cloud account](/docs/guides/cloud-setup#cloud-untrusted)</CloudContent> to authenticate with your API key.
</Admonition>

## Update the experiment
<LegacyContent>
<Tabs>
  <TabItem value="LocalExperiment" label="Local Experiment">
```python
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit.random import random_circuit

qc_random = [(random_circuit(20, 20, measure=True)) for _ in range(30)]
optimization_level = 3

service = QiskitRuntimeService()
backend = service.backend(backend_name)

pass_manager = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend
)

# @distribute_task(target={"cpu": 1})
def transpile_parallel(circuit, pass_manager):
    """Distributed transpilation for an abstract circuit into an ISA circuit for a given backend."""

    isa_circuit = pass_manager.run(circuit)

    return isa_circuit

transpiled_circuits = [
    transpile_parallel(circuit, pass_manager)
    for circuit in qc_random
]

print(transpiled_circuits)
```
  </TabItem>

  <TabItem value="Serverless" label="Serverless">
```python
# transpile_remote.py

from qiskit.transpiler import generate_preset_pass_manager
from qiskit_serverless import get_arguments, save_result, distribute_task, get, update_status, Job
from qiskit_ibm_runtime import QiskitRuntimeService

# Get program arguments
arguments = get_arguments()
circuits = arguments.get("circuits")
backend_name = arguments.get("backend_name")
optimization_level = arguments.get("optimization_level")
pass_manager = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_name
)

# Distribute task across workers
@distribute_task(target={"cpu": 1})
def transpile_parallel(circuit, pass_manager):
    """Distributed transpilation for an abstract circuit into an ISA circuit for a given backend."""

    isa_circuit = pass_manager.run(circuit)

    return isa_circuit

try:
    # Get backend
    service = QiskitRuntimeService()
    backend = service.get_backend(backend_name)

    # run distributed tasks as async function
    # we get task references as a return type
    update_status(Job.OPTIMIZING_HARDWARE)
    sample_task_references = [
        transpile_parallel(circuit, pass_manager)
        for circuit in circuits
    ]

    # now we need to collect results from task references
    results = get(sample_task_references)

    # Return results
    save_result({
        "transpiled_circuits": results
    })

except Exception as e:
    # Exception handling
    import traceback
    print(traceback.format_exc())
```

  </TabItem>
</Tabs>
</LegacyContent>
<CloudContent>

<Tabs>
  <TabItem value="LocalExperiment" label="Local Experiment">
```python
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit.random import random_circuit

qc_random = [(random_circuit(20, 20, measure=True)) for _ in range(30)]
optimization_level = 3

service = QiskitRuntimeService(channel="ibm_quantum_platform")
backend = service.get_backend(backend_name)

pass_manager = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend
)

# @distribute_task(target={"cpu": 1})
def transpile_parallel(circuit, pass_manager):
    """Distributed transpilation for an abstract circuit into an ISA circuit for a given backend."""

    isa_circuit = pass_manager.run(circuit)

    return isa_circuit

transpiled_circuits = [
    transpile_parallel(circuit, pass_manager)
    for circuit in circuits
]

print(transpiled_circuits)
```
  </TabItem>

  <TabItem value="Serverless" label="Serverless">
    ```python
# transpile_remote.py

from qiskit.transpiler import generate_preset_pass_manager
from qiskit_serverless import get_arguments, save_result, distribute_task, get
from qiskit_ibm_runtime import QiskitRuntimeService

# Get program arguments
arguments = get_arguments()
circuits = arguments.get("circuits")
backend_name = arguments.get("backend_name")
optimization_level = arguments.get("optimization_level")
pass_manager = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_name
)

# Distribute task across workers
@distribute_task(target={"cpu": 1})
def transpile_parallel(circuit, pass_manager):
    """Distributed transpilation for an abstract circuit into an ISA circuit for a given backend."""

    isa_circuit = pass_manager.run(circuit)

    return isa_circuit

try:
    # Get backend
    service = QiskitRuntimeService()
    backend = service.get_backend(backend_name)

    # run distributed tasks as async function
    # we get task references as a return type
    sample_task_references = [
        transpile_parallel(circuit, pass_manager)
        for circuit in circuits
    ]

    # now we need to collect results from task references
    results = get(sample_task_references)

    # Return results
    save_result({
        "transpiled_circuits": results
    })

except Exception as e:
    # Exception handling
    import traceback
    print(traceback.format_exc())
```

  </TabItem>
</Tabs>
</CloudContent>

## Upload to Qiskit Serverless

Follow the instructions on the [Introduction to Qiskit Functions](/docs/guides/functions) page to authenticate with your API key.

```python
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction

# Authenticate to the remote cluster and submit the pattern for remote execution.
serverless = QiskitServerless()

transpile_remote_demo = QiskitFunction(
    title="transpile_remote_serverless",
    entrypoint="transpile_remote.py",
    working_dir="./source_files/",
)

serverless.upload(transpile_remote_demo)
```

Output
```text
'transpile_remote_serverless'
```

## Remotely run in Qiskit Serverless

```python
from qiskit.circuit.random import random_circuit
from qiskit_ibm_runtime import QiskitRuntimeService

# Setup inputs
qc_random = [(random_circuit(20, 20, measure=True)) for _ in range(30)]
backend = "ibm_brisbane"
optimization_level = 3

# Running program
transpile_remote_serverless = serverless.load('transpile_remote_serverless')
job = transpile_remote_serverless.run(
    circuits=qc_random,
    backend=backend,
    optimization_level=optimization_level
)

job.job_id
```
Output
```text
'727e921d-512d-4b7d-af97-fe29e93ce7ea'
```

## Next steps

<Admonition type="info" title="Recommendations">

- Read a paper in which researchers used Qiskit Serverless and quantum-centric supercomputing to [explore quantum chemistry](https://arxiv.org/abs/2405.05068v1).

</Admonition>